In [1]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random
import itertools
import os, copy
from copy import deepcopy
import time

def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard

def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex

def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

In [2]:
# Form groups and calculate nr of members
def EvalBoard(board):
    score = find_scores(board, 'b') #score of player1
    score2 = find_scores(board, 'w') #score of player2

    group = findGroups(score)
    group2 = findGroups(score2)

    player_one_total, player_two_total = result(group, group2)
    tempscore = 0
  
    return player_one_total, player_two_total, player_one_total - player_two_total

def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

def listOfEmptySpots(board):
    return [k for k, v in board.items() if v == '']

def neighbours(tiles, scoregroup, visited):
    group = []
    for tile in tiles:
        for direction in hex_directions:
            new_tile2 = tuple(map(sum, zip(tile, direction)))
            if new_tile2 in scoregroup and new_tile2 not in visited:
                group += [new_tile2]
                visited += [new_tile2]
    return group, visited

def findGroups(score):
    newlist = []
    visited = []
    for sc in score:
        temp = [sc]
        if sc not in visited:
            visited += [sc]
            groupt, visited = neighbours([sc], score, visited)
            temp += groupt
            visited += groupt
            find = True
            while find:
                groupt, visited = neighbours(groupt, score, visited)
                if len(groupt) != 0:
                    temp += groupt
                else:
                    find = False
                    
            newlist += [len(temp)]
    return newlist


# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1
    return number_ofstones >= number_ofcells -1

#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

In [3]:
def moves_left(board):
    print('These are the moves left: ')
    
    moves = ''
    counter = 0
    for pos in board:
        
        if board[pos] == '':
            moves += str(pos)
            counter += 1
            if counter == 7:
                moves += '\n'
                counter = 0
            else:
                moves += ' '
    print (moves)

In [4]:
def printBoard(board):
    for x in range(-N, N+1):
        string = ''
        if x > 0:
            string += ' ' * (N-(N-x))
        else:
            string += ' ' * -(N-(N-x))
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):

            if (x, y, -x-y) in board:
                if board[(x, y, -x-y)] == '':
                    string += '_ '
                else:
                    string += board[(x, y, -x-y)] + ' '
        print(string)      

In [5]:
def zobristStateScore(board):

    # Create a zobrist key for the new board/childnode
    zidentity = boardToZobrist(board, zboard, wzobristboard, bzobristboard)
    game_state, maxplayer, minplayer, tempscore = 0, 0, 0, 0
    # If the board is full, tell the score
    if fullBoard(board):

        score = find_scores(board, 'b') #score of player1
        score2 = find_scores(board, 'w') #score of player2

        count_list = findGroups(score)
        count_list2 = findGroups(score2)

        player_one_total, player_two_total = result(count_list, count_list2)

        # Did player 1 win, lose or draw?
        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]
        tempscore = ((-999, 999)[player_one_total < player_two_total ], -499)[player_one_total == player_two_total]
    else:
        maxplayer, minplayer, tempscore = EvalBoard(board)
        
    return zidentity%1000, game_state, tempscore, maxplayer, minplayer

In [6]:
random.seed(1356) # set seed so that each play is the same

hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 4 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4

board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

INFINITY = 999

In [7]:
def AlphaBeta(board, depth, alpha, beta, MAX):
    if depth == 0 and fullBoard(board):
        if MAX:
            return EvalBoard(board)[0]
        else:
            return -EvalBoard(board)[1]
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
#     print('Alpha-Beta', alpha, beta, EvalBoard(board))
    if MAX:
        bestValue = -INFINITY
        for cell in all_cells:
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'b')
                makeMove(board_temp, [x2, y2], 'w')
                bestValue = max(bestValue, AlphaBeta(board_temp, depth-1, alpha, beta, False))
                
                alpha = max(alpha, bestValue)
                if beta <= alpha:
                    break # beta cut-off
        return bestValue
    else: # MIN
        bestValue = INFINITY
        for cell in all_cells:
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'b')
                makeMove(board_temp, [x2, y2], 'w')
                bestValue = min(bestValue, AlphaBeta(board_temp, depth-1, alpha, beta, True))
                
                beta = min(beta, bestValue)
                if beta <= alpha:
                    break # alpha cut-off
        return bestValue

In [8]:
def AlphaBetaTT(board, depth, alpha, beta, MAX):
    if depth == 0 or fullBoard(board):
#         if MAX:
#             return EvalBoard(board)[0]
#         else:
        return EvalBoard(board)[2]
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
#     print('entered', depth)
    if MAX:
        v = -INFINITY
        for cell in all_cells:
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'b')
                makeMove(board_temp, [x2, y2], 'w')
                
                boardZobrist = boardToZobrist(board_temp, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000 #% len(tt)
#                 print('Zobrist', index, boardZobrist)
                if index in tt:
                    continue
                
                index, state, score, maxplayer, _ = zobristStateScore(board_temp)
#                 print('Max', index, state, maxplayer)
                tt[index] = [depth, state, maxplayer]
                value = AlphaBetaTT(board_temp, depth-1, alpha, beta, False)
                v = max(v, value)
                alpha = max(alpha, v)
                
                if beta <= alpha:
                    break # beta cut-off
        return v
    else: # MIN
        v = INFINITY
        for cell in all_cells:
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'b')
                makeMove(board_temp, [x2, y2], 'w')
                
                #
                # Search for the key in the transposition table
                # 
                boardZobrist = boardToZobrist(board_temp, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000 #% len(tt)
#                 print('Zobrist', index, boardZobrist)
                if index in tt:
                    continue

                index, state, score, _, minplayer = zobristStateScore(board_temp)
#                 print('Min', index, state, minplayer)
                tt[index] = [depth, state, minplayer]
            
                value = AlphaBetaTT(board_temp, depth-1, alpha, beta, True)
                v = min(v, value)
                
                beta = min(beta, v)
                if beta <= alpha:
                    break # alpha cut-off
        return v

In [9]:
def NegamaxAB(board, depth, alpha, beta, side):
    if depth == 0 or fullBoard(board):
        return side * EvalBoard(board)[2]
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
    
    v = -INFINITY
    for cell in all_cells:
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
            makeMove(board_temp, [x, y], 'b')
            makeMove(board_temp, [x2, y2], 'w')
            v = max(v, -NegamaxAB(board_temp, depth-1, -alpha, -beta, -side))
            
            alpha = max(alpha, v)
            if beta <= alpha:
                break # beta cut-off
    return v

In [10]:
def Negascout(board, depth, alpha, beta, side):
    if depth == 0 or fullBoard(board):
        return side * EvalBoard(board)[2]
    
    first_child = True
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
    
    for cell in all_cells:
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]
        
        #Make two valid moves
        if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
            makeMove(board_temp, [x, y], 'b')
            makeMove(board_temp, [x2, y2], 'w')
            
            if not first_child:
                score = -Negascout(board_temp, depth-1, -alpha-1, -alpha, -side)
                if alpha < score and score < beta:
                    score = -Negascout(board_temp, depth-1, -beta, -score, -side)
            else:
                first_child = False
                score = -Negascout(board_temp, depth-1, -beta, -alpha, -side)
                
            alpha = max(alpha, score)
            if alpha >= beta:
                break
    return alpha

In [11]:
def BestMove(board):
    maxPoints = 0
    mx = N+1; 
    my = N+1
    mx2 =  N+1; 
    my2 = N+1
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
    
    for cell in all_cells:

        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):

            makeMove(board_temp, [x, y], 'b')
            makeMove(board_temp, [x2, y2], 'w')

            if opt == 0:
                points, _, _, = EvalBoard(board_temp)
            elif opt == 1:
                points = NegamaxAB(board_temp, depth, -INFINITY, +INFINITY, 1)
            elif opt == 2:
                points = Negascout(board_temp, depth, -INFINITY, +INFINITY, 1)
            elif opt == 3:
                points = AlphaBeta(board_temp, depth, -INFINITY, +INFINITY, True)
            elif opt == 4:
#                 print('Chosen')
                #
                # Search for the key in the transposition table
                # 
                boardZobrist = boardToZobrist(board, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000
#                 print('Zobrist', index, boardZobrist)
                if index in tt:
                    points = tt[index][-1]
                
                    if points > maxPoints:
                        maxPoints = points
                        mx = x; my = y
                        mx2 = x2; my2 = y2

                        continue
                points = AlphaBetaTT(board_temp, depth, -INFINITY, +INFINITY, True)

#             print(points, maxPoints)
            if points > maxPoints:
                maxPoints = points
                mx = x; my = y
                mx2 = x2; my2 = y2
    
    
    return [mx, my], [mx2, my2], board_temp

In [12]:
# AlphaBetaTT(board, depth, -INFINITY, +INFINITY, True)

In [13]:
# def startGame(play):
tt = {}
points = 0
print ('Omega game initialised')
ai = ['Evaluate board', 'Negamax', 'Negascout(PVS)', 'Alpha Beta regular', 'Alpha Beta TT']
menu = ['0: EvalBoard', '1: Negamax w/ Alpha-Beta Pruning', '2: Negascout (Principal Variation Search)', 
        '3: Alpha Beta regular', '4: Alpha Beta TT']

for item in menu:
    print(item)
    
opt = int(input('Select AI engine: '))
if not (opt >= 0 and opt <= 4):
    opt = int(input(str(opt) + ' is the wrong input, select your AI again. press 0, 1, 2, 3 or 4'))
else:
    print('You have selected the', ai[opt], 'AI')
print ('\n1: User\n2: AI \n(Press Enter for Exit!)')
board = initializeBoard()
player = 3
# player = 1
depth = 7

players = ['', 'human', 'ai', 'ai', 'ai']

aiVShuman = 1
# aiVShuman = 0

play = True

while play:
    print()
#         PrintBoard()
    print ('The next move is for the player', players[player])

    if fullBoard(board):
        print ('No moves left! Game ended!')
        print ('Score User: ' + str(EvalBoard(board) [0]))
        print ('Score AI  : ' + str(EvalBoard(board) [1]))
        if EvalBoard(board) [0] > EvalBoard(board) [1]:
            print('Player b wins')
        elif EvalBoard(board) [0] == EvalBoard(board) [1]:
            print('It\'s a tie!')  
        else:
            print('AI wins')
#             os._exit(0)
        play = False

    if player == 1: # player
        firstmove = True
        while True:
            
            
  
            if firstmove:
                xy = input('X,Y: ')
                if xy == '': os._exit(0)
                    
                # Check available moves
                if xy == '+': 
                    unmakeMove(board, [int(x), int(y)], 'b')
                    unmakeMove(board, [int(x2), int(y2)], 'w')
                    print('Undone', [int(x), int(y)], [int(x2), int(y2)])
                    xy = input('X,Y: ')

                # Check available moves
                if xy == '?': 
                    moves_left(board)
                    xy = input('X,Y: ')

                (x, y) = xy.split(',')
                x = int(x)
                y = int(y)
                done = 0
                if validMove(board, x, y):
                    board = makeMove(board, [x, y], 'b')
                    player = 2
                    done = 1
                    firstmove = False
                else:
                    print ('Invalid move! Try again2!')
                    continue

            else:
                xy = input('X,Y: ')
                if xy == '': os._exit(0)
                    
                # Check available moves
                if xy == '+': 
                    unmakeMove(board, [int(x), int(y)], 'b')
                    unmakeMove(board, [int(x2), int(y2)], 'w')
                    print('Undone', [int(x), int(y)], [int(x2), int(y2)])
                    xy = input('X,Y: ')

                # Check available moves
                if xy == '?': 
                    moves_left(board)
                    xy = input('X,Y: ')

                (x, y) = xy.split(',')
                x2 = int(x)
                y2 = int(y)

                if done == 1 and validMove(board, x2, y2):
                    board = makeMove(board, [x2, y2], 'w')
                    player = 2
                    printBoard(board) 
                    break
                else:
                    print ('Invalid move! Try again!')
    else: # AI
#         if player == 2:
        # Check available moves
    
        # Check available moves
#         if xy == '+': 
#             unmakeMove(board, moveb, 'b')
#             unmakeMove(board, movew, 'w')
#             print('Undone', moveb, movew)
#             xy = input('X,Y: ')
                
        start = time.time()
        moveb, movew, _ = BestMove(board)
        print('Thinking time: ', time.time() - start)
        
        if moveb[0] != N +1:
            makeMove(board, moveb, 'b')
            makeMove(board, movew, 'w')
            
            if player == 3 and not aiVShuman:
                player = 4
            elif player == 4 and not aiVShuman:
                player = 3
            elif player == 2 and not aiVShuman:
                player = 1
            elif player == 3 and aiVShuman:
                player = 1
            elif player == 2 and aiVShuman:
                player = 1
            elif player == 4 and aiVShuman:
                player = 1
              
            print ('AI played b (X Y): ', moveb, 'and w (X Y): ', movew)
            printBoard(board) 
        else:
            continue
    
if not play:
    print('\nGame ended!')

Omega game initialised
0: EvalBoard
1: Negamax w/ Alpha-Beta Pruning
2: Negascout (Principal Variation Search)
3: Alpha Beta regular
4: Alpha Beta TT
Select AI engine: 4
You have selected the Alpha Beta TT AI

1: User
2: AI 
(Press Enter for Exit!)

The next move is for the player ai
Thinking time:  0.21778106689453125
AI played b (X Y):  [-4, 0] and w (X Y):  [-4, 1]
    b w _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ _ _ _ 

The next move is for the player human
X,Y: 0,0
X,Y: 1,0
    b w _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
_ _ _ _ b _ _ _ _ 
 _ _ _ _ w _ _ _ 
  _ _ _ _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ _ _ _ 

The next move is for the player ai
Thinking time:  0.24858403205871582
AI played b (X Y):  [4, -2] and w (X Y):  [4, -1]
    b w _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
_ _ _ _ b _ _ _ _ 
 _ _ _ _ w _ _ _ 
  _ _ _ _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ b w 

X,Y: ?
These are the moves left: 
(-2, 2, 0) (-2, 3, -1) (-2, 4, -2) (-1, -3, 4) (-1, -2, 3) (-1, -1, 2) (-1, 0, 1)
(-1, 1, 0) (-1, 2, -1) (-1, 3, -2) (-1, 4, -3) (0, -4, 4) (0, -3, 3) (0, -2, 2)
(0, -1, 1) (0, 1, -1) (0, 2, -2) (0, 4, -4) (1, -4, 3) (1, -3, 2) (1, -2, 1)
(1, -1, 0) (3, -1, -2) 
X,Y: -2,2
X,Y: -2,3
    b w b b w 
   b w b w b w 
  b w b w b w _ 
 _ _ _ _ _ _ _ _ 
_ _ _ _ b _ _ w _ 
 _ _ _ _ w b w b 
  w b w b w w b 
   w b w _ b w 
    b w b w b 

The next move is for the player ai
Thinking time:  0.08640527725219727
AI played b (X Y):  [-2, 4] and w (X Y):  [-1, -3]
    b w b b w 
   b w b w b w 
  b w b w b w b 
 w _ _ _ _ _ _ _ 
_ _ _ _ b _ _ w _ 
 _ _ _ _ w b w b 
  w b w b w w b 
   w b w _ b w 
    b w b w b 

The next move is for the player human
X,Y: -2,4
Invalid move! Try again2!
X,Y: ?
These are the moves left: 
(-1, -2, 3) (-1, -1, 2) (-1, 0, 1) (-1, 1, 0) (-1, 2, -1) (-1, 3, -2) (-1, 4, -3)
(0, -4, 4) (0, -3, 3) (0, -2, 2) (0, -1, 1) (0, 1, -1) (0, 2, -2) (

# 

In [14]:
printBoard(board) 

    b w b b w 
   b w b w b w 
  b w b w b w b 
 w b w b w b w b 
w w _ b b w b w w 
 b w b w w b w b 
  w b w b w w b 
   w b w b b w 
    b w b w b 
